In [1]:
import numpy as np
import dipy.data as dpd
import dipy.core.sphere as dps 
import dipy.sims as sims
import dipy.core.gradients as grad
import dipy.core.geometry as geo

In [2]:
s1 = dpd.get_sphere()

In [3]:
s1.vertices.shape

(362, 3)

In [4]:
def l2norm(vector):
    """ 
    
    
    """
    return vector / np.dot(vector, vector)
    

In [5]:
import dipy.reconst.dti as dti

In [6]:
def single_tensor(evecs, evals, bvec, bval=1, S0=1):
    R = np.asarray(evecs)
    D = np.dot(np.dot(R, np.diag(evals)), R.T)
    return S0 * np.exp(-bval * np.dot(np.dot(bvec, D), bvec.T))

In [23]:
out_dir=np.array([1, 0, 0]) 
evals=np.array([1.5, 0.5, 0.5]) 
bval=1

In [29]:
def out_signal(out_dir, evals=np.array([1.5, 0.5, 0.5]), bval=1):
    # Rotate the canonical tensor towards the output direction and  
    # calculate the signal you would have gotten in the direction
    canon_tensor = np.array([[evals[0], 0, 0], [0, evals[1], 0], [0, 0, evals[2]]])
    rot_matrix = geo.vec2vec_rotmat(np.array([1,0,0]), out_dir)
    out_tensor = np.dot(rot_matrix, canon_tensor)
    evals, evecs = dti.decompose_tensor(out_tensor)
    return single_tensor(evecs, evals, out_dir, bval=bval, S0=1)

In [30]:
evals

array([ 1.5,  0.5,  0.5])

In [31]:
evecs

array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.]])

In [32]:
single_tensor(evecs, evals, out_dir, bval=bval, S0=1)

0.22313016014842982

In [33]:
out_dir

array([1, 0, 0])

In [43]:
def design_signal(location, out_dir, in_dir, evals=np.array([1.5, 0.5, 0.5]), bval=1):

    """
    location : the center-to-center location (relative to 0,0,0)
    
    in_dir : observations
    out_dir : parameters
    
    
    """ 
    this_signal = out_signal(out_dir, evals=np.array([1.5, 0.5, 0.5]), bval=1)
    norm_location = l2norm(location)
    out_corr = np.dot(norm_location, out_dir)
    return out_corr * this_signal 
    
    

In [44]:
def test_out_signal():
    sig1 = out_signal(np.array([1, 0, 0]), np.array([1.5, 0.5, 0.5]), 1)
    sig2 = single_tensor(np.eye(3), np.array([1.5, 0.5, 0.5]), np.array([1, 0, 0])) 
    assert sig1 == sig2
              
test_out_signal()

In [42]:
def test_out_signal():
    sig1 = design_signal(np.array([1,0,0]), np.array([1,0,0]), np.array([1,0,0]))
    sig2 = out_signal(np.array([1, 0, 0]), np.array([1.5, 0.5, 0.5]), 1)
    assert sig1 == sig2

test_out_signal()

In [ ]:
def design_matrix(sphere, bvecs):
    """ 

    """
    
    